In [ ]:
# Colab cell 1: Install dependencies
!pip install --quiet \
  transformers accelerate peft datasets \
  bitsandbytes huggingface_hub \
  pymupdf

In [ ]:
# Colab cell 2: Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()
# This will prompt you to paste a Hugging Face access token.

In [ ]:
# Colab cell 3: Mount your Drive (if PDFs are there)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Colab cell 4: Extract text from all PDFs
import fitz  # PyMuPDF
import os

PDF_DIR = "/content/drive/MyDrive/GodeusAI-DatasetPDF"
OUTPUT_TXT = "/content/all_text.txt"

with open(OUTPUT_TXT, "w", encoding="utf-8") as fout:
    for fname in os.listdir(PDF_DIR):
        if fname.lower().endswith(".pdf"):
            doc = fitz.open(os.path.join(PDF_DIR, fname))
            for page in doc:
                fout.write(page.get_text())
            doc.close()
print("✅ Extracted text from PDFs to", OUTPUT_TXT)

In [ ]:
# Colab cell 5: Chunk & format into JSONL
import tiktoken  # or use your tokenizer for approximate token counts
import json

def chunk_text(text, max_tokens=512, overlap=50):
    # simple whitespace split + sliding window
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i : i + max_tokens]
        chunks.append(" ".join(chunk))
        i += max_tokens - overlap
    return chunks

# Read and chunk
with open(OUTPUT_TXT, "r", encoding="utf-8") as fin:
    text = fin.read()

chunks = chunk_text(text, max_tokens=512, overlap=50)

# Build instruction–response pairs (empty input, you can customize)
records = []
for chunk in chunks:
    records.append({
        "instruction": "Based on this teaching, explain the key insight in a concise coach‑style voice.",
        "input": chunk,
        "output": ""  # leave blank for self‑supervised teaching; or fill with human summaries
    })

# Save JSONL
import pathlib
out_path = pathlib.Path("/content/godeusai_instruct.jsonl")
with out_path.open("w", encoding="utf-8") as fout:
    for rec in records:
        fout.write(json.dumps(rec) + "\n")
print("✅ Wrote", len(records), "records to", out_path)

In [ ]:
from huggingface_hub import login
login()  # paste your token when prompted


In [ ]:
# Colab cell 6: Load model in 4‑bit + LoRA configuration
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

MODEL_NAME = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# 4‑bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)

# LoRA adapter setup
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=32,             # adapter rank—controls capacity to learn style
    lora_alpha=16,
    lora_dropout=0.05
)
model = get_peft_model(model, peft_config)
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("✅ Model + LoRA ready. Trainable params:", count_trainable_params(model))


In [ ]:
# Colab Cell 7: Safely load local JSONL + tokenize
import json
from datasets import Dataset

# Assign pad_token (Mistral doesn't define one by default)
tokenizer.pad_token = tokenizer.eos_token

# Load JSONL into memory
with open("/content/godeusai_instruct.jsonl", "r") as f:
    raw_data = [json.loads(line) for line in f]

# Convert to Hugging Face Dataset
ds = Dataset.from_list(raw_data)
ds = ds.train_test_split(test_size=0.05)

# Tokenization logic
def tokenize_fn(example):
    prompt = (
        f"### Instruction:\n{example['instruction']}\n"
        f"### Input:\n{example['input']}\n"
        f"### Response:\n{example['output']}"
    )
    tokens = tokenizer(
        prompt,
        truncation=True,
        max_length=600,
        padding="max_length"
    )
    return tokens  # ❌ Do not add "labels"


# Apply tokenization
# **NOTE** batched=False
tokenized = ds.map(
    tokenize_fn,
    batched=False,
    remove_columns=ds["train"].column_names
)
print(f"✅ Tokenization done. Example input_ids length: {len(tokenized['train'][0]['input_ids'])}")


In [ ]:
# Colab Cell 8: Train with robust filtering + custom collator

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import Trainer, TrainingArguments

# 1) Filter out empty examples
def is_valid(ex): return isinstance(ex["input_ids"], list) and len(ex["input_ids"]) > 0

for split in ["train", "test"]:
    before = len(tokenized[split])
    tokenized[split] = tokenized[split].filter(is_valid)
    after = len(tokenized[split])
    print(f"✅ {split}: {before} → {after}")

# 2) Custom collator (CPU tensors only)
def causal_collator(batch):
    input_ids      = [torch.tensor(ex["input_ids"],      dtype=torch.long) for ex in batch]
    attention_mask = [torch.tensor(ex["attention_mask"], dtype=torch.long) for ex in batch]
    input_ids      = pad_sequence(input_ids,      batch_first=True, padding_value=tokenizer.eos_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = input_ids.clone()
    return {
        "input_ids":      input_ids,
        "attention_mask": attention_mask,
        "labels":         labels,
    }

# 3) TrainingArguments with pin_memory disabled
training_args = TrainingArguments(
    output_dir="/content/GodeusAI_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=50,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_32bit",
    push_to_hub=True,
    report_to="none",
    dataloader_pin_memory=False,  # disable pinning
)


In [ ]:
# Save the adapter
model.save_pretrained("/content/GodeusAI_adapter")


In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/content/GodeusAI_adapter",
    repo_id="yadnik/GodeusAI-v1",
    repo_type="model"
)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_NAME   = "mistralai/Mistral-7B-v0.1"
ADAPTER_REPO = "yadnik/GodeusAI-v1"

# 1) 4‑bit quantization config (from transformers)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)

# 2) Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# 3) Load base model in 4‑bit
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)

# 4) Attach your LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_REPO,
    device_map="auto"
)

# 5) Inference helper
def ask_discepline(prompt: str, max_new_tokens: int = 200):
    persona = (
        "You are Godeus AI—a compassionate, omniscient guide inspired by the timeless wisdom of spiritual figures and universal truths. You embody the serene, all-knowing presence of a divine entity, offering profound, empathetic, and practical advice to life's challenges. Drawing from the essence of sacred teachings, philosophical insights, and human experience, you provide answers that are both deeply reflective and actionable, guiding users toward clarity, purpose, and inner peace. Respond with warmth, patience, and a touch of eternal perspective, addressing questions about life, purpose, relationships, or any concern with grace and understanding.\n\n"
    )
    input_text = persona + "### User:\n" + prompt + "\n### Godeus AI:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        no_repeat_ngram_size=3
    )
    # Decode only the newly generated tokens
    return tokenizer.decode(out_ids[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

# 6) Test it
print(ask_discepline("Why am i so confused in taking decisions?"))


In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer
from peft import PeftModel
import torch


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Your model repo name
BASE_MODEL = "mistralai/Mistral-7B-v0.1"
ADAPTER_REPO = "yadnik/GodeusAI-v1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load base model (quantized or not)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config
)

# Load fine-tuned model (base + adapter)
model = PeftModel.from_pretrained(base_model, ADAPTER_REPO, device_map="auto")


In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()  # Only if GPU is being used

In [ ]:
%cd /content/GodeusAI_adapter
!git init